# MapReduce Graduaciones

In [ ]:
###############################################################
#!/usr/bin/python
# Purpose: Migrate Products from MySQL to Mongo
# Dependecies : python-pip , pymongo , mysqldb
# apt-get install python-pip ; pip install pymongo;
# apt-get install python-MySQLdb


import pymongo
import datetime
import enum
from pprint import pprint

In [2]:
mongodb_host = "mongodb://localhost:11111/"
mongodb_dbname = "project_test"

mclient = pymongo.MongoClient(mongodb_host)
mdb = mclient[mongodb_dbname] 

In [4]:
collections = mdb.list_collection_names()
print(collections)

['publicacion', 'cargos', 'registro_publicaciones', 'sede_universidad', 'empresa', 'facultad', 'departamento', 'time_', 'egresado', 'contrataciones', 'programa', 'graduaciones', 'institucion', 'revista']


In [7]:
contrataciones = mdb["contrataciones"]
pprint(contrataciones.find()[0])

{'_id': ObjectId('6626a46aa62db005d03b9ddf'),
 'cargo_id': {'cargo_id': 1478,
              'cargo_nombre': 'GEOPHYSICIST/FIELD SEISMOLOGIST'},
 'contratacion_id': 1,
 'date_id': datetime.datetime(1981, 5, 23, 0, 0),
 'egresado_id': {'egresado_address': '7090 John Lodge\nWest Toddtown, NM 23637',
                 'egresado_cellphone': '710-374-63',
                 'egresado_edad': 33,
                 'egresado_emailaddress': 'xford@savage.com',
                 'egresado_etnicgroup': 'ACTIVO',
                 'egresado_firstlastname': 'BURTON',
                 'egresado_gender': 'MASCULINO',
                 'egresado_id': 32,
                 'egresado_names': 'ASHLEY',
                 'egresado_secondlastname': 'SCHROEDER'},
 'empresa_id': {'empresa_id': 76,
                'empresa_nombre': 'DAVIS LLC',
                'empresa_origen': 'BERMUDA'},
 'salario': 44947741.0}


In [9]:
# Egresados por facultad

stage_group_facultad = {
  "$group": {
    "_id" : "$empresa_id.empresa_origen",
    "count_egresado": {"$sum": 1}
  }
}

stage_out_origen = {
  "$out": "contrataciones_origen"
}

pipeline = [
  stage_group_facultad,
  stage_out_origen
]

results = contrataciones.aggregate(pipeline)

for graduacion_mapreduce in mdb["contrataciones_origen"].find():
  print(graduacion_mapreduce)

{'_id': 'VENEZUELA', 'count_egresado': 2}
{'_id': 'ZIMBABWE', 'count_egresado': 2}
{'_id': 'BELGIUM', 'count_egresado': 4}
{'_id': "COTE D'IVOIRE", 'count_egresado': 2}
{'_id': 'HAITI', 'count_egresado': 2}
{'_id': 'PAPUA NEW GUINEA', 'count_egresado': 2}
{'_id': 'MAURITANIA', 'count_egresado': 2}
{'_id': 'SAINT KITTS AND NEVIS', 'count_egresado': 4}
{'_id': 'ESTONIA', 'count_egresado': 4}
{'_id': 'SRI LANKA', 'count_egresado': 2}
{'_id': 'SAO TOME AND PRINCIPE', 'count_egresado': 6}
{'_id': 'ALBANIA', 'count_egresado': 6}
{'_id': 'BULGARIA', 'count_egresado': 2}
{'_id': 'JORDAN', 'count_egresado': 2}
{'_id': 'BOTSWANA', 'count_egresado': 2}
{'_id': 'PAKISTAN', 'count_egresado': 2}
{'_id': 'POLAND', 'count_egresado': 2}
{'_id': 'GERMANY', 'count_egresado': 10}
{'_id': 'AUSTRIA', 'count_egresado': 2}
{'_id': 'BHUTAN', 'count_egresado': 8}
{'_id': 'KIRIBATI', 'count_egresado': 2}
{'_id': 'TAIWAN', 'count_egresado': 2}
{'_id': 'REUNION', 'count_egresado': 2}
{'_id': 'ANDORRA', 'count_egre

In [11]:
# Egresados por tipo programa

stage_group_empresa_nombre = {
  "$group": {
    "_id" : "$empresa_id.empresa_nombre",
    "count_egresado": {"$sum": 1}
  }
}

stage_out_empresa = {
  "$out": "contrataciones_por_empresa"
}

pipeline = [
  stage_group_empresa_nombre,
  stage_out_empresa
]

results = contrataciones.aggregate(pipeline)

for contrataciones_mapreduce in mdb["contrataciones_por_empresa"].find():
  print(contrataciones_mapreduce)

{'_id': 'ADAMS-WELLS', 'count_egresado': 2}
{'_id': 'POWELL-WARD', 'count_egresado': 2}
{'_id': 'PHILLIPS LLC', 'count_egresado': 4}
{'_id': 'HODGES LLC', 'count_egresado': 6}
{'_id': 'RAMIREZ AND SONS', 'count_egresado': 4}
{'_id': 'CERVANTES, CALLAHAN AND OLSON', 'count_egresado': 4}
{'_id': 'STRONG AND SONS', 'count_egresado': 2}
{'_id': 'WILSON-MARTIN', 'count_egresado': 4}
{'_id': 'CLARK-FRITZ', 'count_egresado': 2}
{'_id': 'GONZALEZ, NASH AND CURTIS', 'count_egresado': 2}
{'_id': 'DILLON-POTTER', 'count_egresado': 4}
{'_id': 'PRICE, COOKE AND JAMES', 'count_egresado': 2}
{'_id': 'LEE, KELLY AND LINDSEY', 'count_egresado': 4}
{'_id': 'MORGAN, GREER AND MARQUEZ', 'count_egresado': 2}
{'_id': 'GREEN PLC', 'count_egresado': 4}
{'_id': 'HUDSON-WOOD', 'count_egresado': 4}
{'_id': 'MARTINEZ-BROWN', 'count_egresado': 2}
{'_id': 'PORTER-OROZCO', 'count_egresado': 2}
{'_id': 'SMITH, JOHNSON AND FINLEY', 'count_egresado': 4}
{'_id': 'BROWN, SAUNDERS AND SIMMONS', 'count_egresado': 2}
{'_id'

In [12]:
# Egresados por tipo sede

stage_group_year = {
  "$group": {
    "_id" : {"$year": {"$toDate": "$date_id"}},
    "count_contratados": {"$sum": 1}
  }
}

stage_out_year = {
  "$out": "contrataciones_year"
}

pipeline = [
  stage_group_year,
  stage_out_year
]

results = contrataciones.aggregate(pipeline)

for contrataciones_mapreduce in mdb["contrataciones_year"].find():
  print(contrataciones_mapreduce)

{'_id': 1971, 'count_contratados': 6}
{'_id': 1996, 'count_contratados': 4}
{'_id': 2011, 'count_contratados': 6}
{'_id': 2006, 'count_contratados': 2}
{'_id': 2009, 'count_contratados': 12}
{'_id': 2021, 'count_contratados': 2}
{'_id': 1988, 'count_contratados': 4}
{'_id': 1980, 'count_contratados': 2}
{'_id': 1992, 'count_contratados': 8}
{'_id': 2004, 'count_contratados': 2}
{'_id': 1978, 'count_contratados': 4}
{'_id': 1991, 'count_contratados': 4}
{'_id': 1983, 'count_contratados': 6}
{'_id': 2005, 'count_contratados': 4}
{'_id': 2016, 'count_contratados': 4}
{'_id': 1997, 'count_contratados': 4}
{'_id': 1990, 'count_contratados': 6}
{'_id': 2023, 'count_contratados': 8}
{'_id': 1976, 'count_contratados': 2}
{'_id': 2012, 'count_contratados': 4}
{'_id': 1998, 'count_contratados': 6}
{'_id': 2001, 'count_contratados': 6}
{'_id': 2017, 'count_contratados': 2}
{'_id': 1994, 'count_contratados': 2}
{'_id': 2015, 'count_contratados': 6}
{'_id': 1999, 'count_contratados': 6}
{'_id': 197